In [16]:
import os
from collections import defaultdict
import itertools
import re
import pandas as pd
from langconv import Converter
from utils import sepSentence, get_source_comments_data
from my_sentiment import *

#### 给语料库里的句子生成一个字典，key是句子，values是这个句子的所有标签列表【（一级，二级，情感）, (一级，二级，情感)……】

In [2]:
def get_sentence_tags_dict(tags):
    """
        tags: 一级，二级，去重段落，情感倾向四列
    """
    tag_dict = {}
    for _, row in tags.dropna(subset=["一级", "二级"]).iterrows():
        for sentence in row["去重段落"].split(","):
            if sentence not in tag_dict:
                tag_dict[sentence] = []
            tag_dict[sentence].append((row["一级"], row["二级"], row["情感倾向"]))
    return tag_dict

### 1.给评论语句打标签

In [3]:
# 生成最终结果的表头
def get_tag_table(dataset, tag_dict):
    columns = []
    lv_tuples = set(itertools.chain(*(((lv1, lv2) for lv1, lv2, _ in lst) for lst in tag_dict.values())))   # 所有可能的一级二级标签
    for lv1, lv2 in lv_tuples:
        if lv2 == "/":
            lv2 = "其他"
        if lv1 in ['精神认同', '人群', '复购', '首购']:
            columns.append((lv1, lv2))
            continue
        for sentiment in ["(正面)", "(负面)"]:
            columns.append((lv1, lv2 + sentiment))
    columns = pd.MultiIndex.from_tuples(sorted(columns))
    tag_table = pd.DataFrame(0, columns=columns, index=dataset.index)
    return tag_table

In [5]:
def get_tags(dataset, tag_dict, tag_table):
    no_tag_comments = []
    no_tag_comments_count = 0
    tag_index = {item: i for i, item in enumerate(tag_table.columns)}
    for i, row in dataset.iterrows():
        sentences = sepSentence(row.loc[pd.IndexSlice["评论内容"]])
        this_tag = []
        is_tagged = None
        for sentence in sentences:
            if sentence not in tag_dict:
                continue
            for lv1, lv2, sentiment in tag_dict[sentence]:
                if lv1 not in ['精神认同', '人群', '复购', '首购']:
                    if lv2 == "/":
                        lv2 = "其他"
                    lv2 = lv2 + ("(正面)" if sentiment == "positive" else "(负面)")
                row = i
                col = tag_index[(lv1, lv2)]
                tag_table.values[row, col] = 1
            is_tagged = True
        if not is_tagged:
            no_tag_comments.append((row['品牌'], row['单品'], row.loc[pd.IndexSlice["评论内容"]]))
            no_tag_comments_count += 1
    print("没有标签评论数占比", no_tag_comments_count / len(dataset))
    # 没有被打上标签的
    pd.DataFrame(no_tag_comments).to_excel(f"model_result/评论标签结果/notagged_comments_result.xlsx", index=False, encoding='utf-8')
    return tag_table

In [6]:
# 保存被打上标签的结果
def save_tagged_result_by_brand(result, save_folder):
    for i, brand in enumerate(result['品牌'].unique()):
        print(i, brand)
        data = result.query('品牌 == @brand')
        if '/' in brand:
            brand = brand.split('/')[1]
        
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        
        save_folder_excel = os.path.join(save_folder,'excel版')
        if not os.path.exists(save_folder_excel):
            os.makedirs(save_folder_excel)

        data.to_excel(os.path.join(save_folder_excel, f"tagged_comments_{brand}_result.xlsx"), index=False, encoding='utf-8')
        data.to_hdf(os.path.join(save_folder, f"tagged_comments_result.h5"), key=f"brand{i}")

In [ ]:
# 读取评论原始数据，非首次
dataset = pd.read_hdf('step_data/comments_dataset.h5', key='src')
# # 首次读取原始评论数据
# dataset = get_source_comments_data(is_first_time=False)
# dataset = dataset.reset_index(drop=True)
# dataset.to_hdf('step_data/comments_dataset.h5', key='src')
# print("原始评价数：", len(dataset))

In [ ]:
# 语料库（带一级二级标签聚类段落）
tags = pd.read_excel(r"model_result/语料库/corpus_final_updated.xlsx")
tags = tags[tags['去重段落'].notnull()]
print(tags['情感倾向'].unique())

In [ ]:
tag_dict = get_sentence_tags_dict(tags)
tag_table = get_tag_table(dataset, tag_dict)
tag_table = get_tags(dataset, tag_dict, tag_table)

In [ ]:
index_columns = ['品牌', '单品', '链接', 'MSRP', '评论序号', '评论内容']
result = pd.concat([dataset[index_columns], tag_table], 1)    # 原始数据有‘男女’列，我的结果没有
tag_columns = set(result.columns) - set(index_columns)        # 所有标签列

In [ ]:
save_tagged_result_by_brand(result, save_folder='model_result/评论标签结果')

In [ ]:
# 燃数除去‘男女’列的标签覆盖率75%
rs_tag_columns = set(dataset.columns) - set(index_columns) - set('男女')
dataset['is_tagged'] = dataset[rs_tag_columns].sum(axis=1) >= 1
dataset['is_tagged'].sum()/len(dataset)

In [ ]:
# 我的结果的覆盖率
result['is_tagged'] = result[tag_columns].sum(axis=1) >= 1
result['is_tagged'].sum() / len(result)

### 2.用第一步带关键词的情感分析给没有标签的补打标签

In [7]:
store = pd.HDFStore("model_result/评论标签结果/tagged_comments_result.h5")
result = pd.DataFrame()
for i in range(len(store.keys())):
    df = store.get(f"/brand{i}")
    result = pd.concat([result, df])
store.close()

In [14]:
index_columns = ['品牌', '单品', '链接', 'MSRP', '评论序号', '评论内容']
tag_columns = set(result.columns) - set(index_columns)

In [15]:
positive_tags = [x for x in tag_columns if '正面' in x[1]]
positive_tags.sort(key=lambda x: x[0])
negative_tags = [x for x in tag_columns if '负面' in x[1]]
negative_tags.sort(key=lambda x: x[0])
other_tags = set(tag_columns) - set(positive_tags) - set(negative_tags)

has_tag_result = result[result["is_tagged"] == True]
has_tag_result.groupby('品牌')[positive_tags].mean().reset_index().to_excel("analysis_data/brand_positive_tags.xlsx", index=False)

In [8]:
no_tag_result = result[result["is_tagged"] == False]
no_tag_result = no_tag_result.reset_index(drop=True)

In [10]:
# 没有标签的评论内容重新断句，生成有关键词、一级、二级、情感倾向的语料库
comments_list = no_tag_result['评论内容'].tolist()
sentences_list = []
for comments in comments_list:
    sentences = sepSentence(comments)
    sentences_list.extend(sentences)
sentiment_result, no_keyword = cal_score(sentences_list, orisen_tag='', keyword_sheet='情感分析关键词补打标签专用')
add_corpus = sentiment_result[sentiment_result['tag_1'].notnull() & (sentiment_result['sentiment'] != '中性')][['tag_1', 'tag_2', 'sentence', 'sentiment']]
add_corpus['sentiment'] = add_corpus['sentiment'].replace({'正面': 'positive', '负面': 'negative'})
add_corpus.columns = ['一级', '二级', '去重段落', '情感倾向']

In [11]:
len(add_corpus)

51987

In [12]:
add_tag_dict = get_sentence_tags_dict(add_corpus)

In [13]:
index_columns = ['品牌', '单品', '链接', 'MSRP', '评论序号', '评论内容']
tag_columns = set(no_tag_result.columns) - set(index_columns) - {'is_tagged'}        # 所有标签列, is_tagged bool类型的影响get_tags函数的效率

In [14]:
tag_table = no_tag_result[tag_columns]

In [15]:
tag_table = get_tags(no_tag_result, add_tag_dict, tag_table)

没有标签评论数占比 0.868903605200749


In [43]:
no_tag_result = pd.concat([no_tag_result[index_columns], tag_table], 1)

C:\Users\rshe11\AppData\Local\Temp\1/ipykernel_17036/2994212450.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  no_tag_result = pd.concat([no_tag_result[index_columns], tag_table], 1)


In [49]:
has_tag_result = result[result["is_tagged"] == True]
updated_result = pd.concat([has_tag_result, no_tag_result])
updated_result['is_tagged'] = updated_result[tag_columns].sum(axis=1) >= 1
updated_result['is_tagged'].sum() / len(updated_result)
save_tagged_result_by_brand(updated_result , save_folder='model_result/评论标签结果补打版')

0 361°


C:\Users\rshe11\Anaconda3\lib\site-packages\pandas\io\pytables.py:314: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]

  f(store)
C:\Users\rshe11\Anaconda3\lib\site-packages\pandas\io\pytables.py:314: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_items] [items->None]

  f(store)
C:\Users\rshe11\Anaconda3\lib\site-packages\pandas\core\generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['品牌', '单品', '链接', 'MSRP', '评论内容'], dtype='object')]

  pytables.to_hdf(


1 Adidas/阿迪达斯


C:\Users\rshe11\Anaconda3\lib\site-packages\pandas\core\generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['品牌', '单品', '链接', 'MSRP', '评论内容'], dtype='object')]

  pytables.to_hdf(


2 Allbirds
3 ANTA/安踏


...

C:\Users\rshe11\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://item.taobao.com/item.htm?&id=656643803142' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\rshe11\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://item.taobao.com/item.htm?&id=653632470647' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\rshe11\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://item.taobao.com/item.htm?&id=653314272569' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\rshe11\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://item.taobao.com/item.htm?&id=652208627822' since it exceeds Excel'

12 Mizuno/美津浓
13 NEW BALANCE
14 Nike/耐克
15 On/昂跑
16 Peak/匹克
17 Puma/彪马
18 Reebok/锐步
19 SALOMON/萨洛蒙
20 Saucony/索康尼
21 Skechers/斯凯奇
22 Under Armour/安德玛
23 XTEP/特步


C:\Users\rshe11\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://item.taobao.com/item.htm?&id=638946061236' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\rshe11\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://item.taobao.com/item.htm?&id=621759396665' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\rshe11\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://item.taobao.com/item.htm?&id=633884474525' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\rshe11\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://item.taobao.com/item.htm?&id=636945001623' since it exceeds Excel'

24 乔丹


C:\Users\rshe11\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://item.taobao.com/item.htm?&id=637706013073' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\rshe11\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://item.taobao.com/item.htm?&id=583070626529' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\rshe11\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://item.taobao.com/item.htm?&id=638285655619' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\rshe11\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://item.taobao.com/item.htm?&id=612718075084' since it exceeds Excel'

In [22]:
a=[x for x in result.columns if '性能' in x[0]]

In [23]:
a

[('性能', '其他(正面)'),
 ('性能', '其他(负面)'),
 ('性能', '回弹(正面)'),
 ('性能', '回弹(负面)'),
 ('性能', '提速(正面)'),
 ('性能', '提速(负面)'),
 ('性能', '缓震(正面)'),
 ('性能', '缓震(负面)'),
 ('性能', '轻量(正面)'),
 ('性能', '轻量(负面)'),
 ('运动性能', '回弹(正面)'),
 ('运动性能', '回弹(负面)'),
 ('运动性能', '提速(正面)'),
 ('运动性能', '提速(负面)')]

In [25]:
a = [(('性能', '回弹(正面)'), ('运动性能', '回弹(正面)')),
     (('性能', '回弹(负面)'), ('运动性能', '回弹(负面)')),
     (('性能', '提速(正面)'), ('运动性能', '提速(正面)')),
     (('性能', '提速(负面)'), ('运动性能', '提速(负面)'))
    ]
for right, wrong in a:
    result[right] =result.loc[:, [right, wrong]].max(axis=1)
    del result[wrong]

In [26]:
result.columns

Index([               '品牌',                '单品',                '链接',
                    'MSRP',              '评论序号',              '评论内容',
            ('人群', '兄弟姐妹'),      ('人群', '其他长辈'),    ('人群', '女朋友/老婆'),
              ('人群', '奶奶'),
       ...
        ('适用场景', '跑跳(负面)'),  ('适用场景', '跳绳(正面)'),  ('适用场景', '跳绳(负面)'),
        ('适用场景', '跳舞(正面)'),  ('适用场景', '跳舞(负面)'), ('适用场景', '马拉松(正面)'),
       ('适用场景', '马拉松(负面)'),      ('首购', '其他首购'),      ('首购', '品牌首购'),
               'is_tagged'],
      dtype='object', length=145)